# Help functions computing moments and Jacobi coefficients

For reference, we used the book 
Gautschi - Orthogonal polynomial, Computation and Approximation

See Table 1.1 page 29 there.

In [1]:
import numpy as np
import scipy

def moment_legendre(n):
    return 1/(1+np.arange(n))

def jacobi_a_legendre(n):
    return 0.5*np.ones( (n,) )

def jacobi_b_legendre(n):
    k = 1.0*np.arange(1, n+1)
    b = 1/( 4*(4-k**(-2)) )
    #b = np.array( [1] + list( b ) )
    return np.sqrt(b)

def moment_laguerre(n):
    k = 1.0*np.arange(0, n)
    k[0] = 1.0
    return np.cumprod(k)

def jacobi_a_laguerre(n):
    k = 1.0*np.arange(1, n+1)
    return 2*k-1

def jacobi_b_laguerre(n):
    k = 1.0*np.arange(1, n+1)
    return k

def moment_chebyshev(n):
    array = np.zeros( (n,))
    array[0] = 1
    for i in range(2, n, 2):
        array[i] = array[i-2]*(i-1)/i
    return array

def jacobi_a_chebyshev(n):
    return np.zeros( (n,) )

def jacobi_b_chebyshev(n):
    array    = np.ones( (n,))*1.0/2
    array[0] = np.sqrt(0.5)
    return array


In [2]:
oprl_setup = 'chebyshev'

if oprl_setup == 'legendre':
    compute_moments  = moment_legendre
    compute_jacobi_a = jacobi_a_legendre
    compute_jacobi_b = jacobi_b_legendre
    scipy_quadrature = scipy.special.roots_legendre
elif oprl_setup == 'laguerre':
    compute_moments  = moment_laguerre
    compute_jacobi_a = jacobi_a_laguerre
    compute_jacobi_b = jacobi_b_laguerre
    scipy_quadrature = scipy.special.roots_laguerre
elif oprl_setup == 'chebyshev':
    compute_moments  = moment_chebyshev
    compute_jacobi_a = jacobi_a_chebyshev
    compute_jacobi_b = jacobi_b_chebyshev
    scipy_quadrature = scipy.special.roots_chebyt

# I. Direct approach: Cholesky

In [3]:
import freeDeconvolution

In [4]:
# Form moment matrix
n = 4
moments_count = 2*n+1
mom_array = compute_moments( moments_count + 1)
print( f'''Computed moments from 0 to 2n+1={len(mom_array)-1}''')

# Compute
#jacobi_a, jacobi_b = freeDeconvolution.oprl.jacobi_from_moments( mom_array )
jacobi_a, jacobi_b = freeDeconvolution.jacobi_from_moments( mom_array, debug=False )

# Print Jacobi coefficients
print( "Computed Jacobi coefficients:")
print( "b: ", jacobi_b )
print( "a: ", jacobi_a )
print( "")

# Given n moments
jacobi_dim = len(jacobi_a)
print( "Theoretical Jacobi coefficients:")
print( "b: ", compute_jacobi_b(jacobi_dim-1) )
print( "a: ", compute_jacobi_a(jacobi_dim) )
print( "")

Computed moments from 0 to 2n+1=9
6-th principal minor is not positive definite
Computed Jacobi coefficients:
b:  [0.70710678 0.5        0.5        0.5       ]
a:  [0. 0. 0. 0. 0.]

Theoretical Jacobi coefficients:
b:  [0.70710678 0.5        0.5        0.5       ]
a:  [0. 0. 0. 0. 0.]



In [5]:
# Quadrature measure from our code
print("Quadrature measure")
support, weights = freeDeconvolution.quadrature_from_jacobi( jacobi_a, jacobi_b)
print( support )
print( weights )
print("")

# Quadrature from scipy
print( "Ground truth")
import scipy
points, weights, mass = scipy_quadrature( len(support), mu=True)
if scipy_quadrature == scipy.special.roots_legendre:
    points = (points+1)/2
weights = weights/weights.sum()
print( points )
print( weights )

Quadrature measure
[-9.51056516e-01 -5.87785252e-01 -1.67520061e-16  5.87785252e-01
  9.51056516e-01]
[0.2 0.2 0.2 0.2 0.2]

Ground truth
[-0.95105652 -0.58778525  0.          0.58778525  0.95105652]
[0.2 0.2 0.2 0.2 0.2]


In [90]:
scenario = 'test2'

if scenario=='test1':
    n_max   = 8
    k       = np.arange(1, n_max+1)
    support = np.cos( (2*k-1)*np.pi/(2*n_max) )
else:
    support = np.array( [1.0, 2.0, 3.0, 5.0, 6.0] )

weights = np.ones( (len(support),) )*1.0/len(support)
np.set_printoptions(precision=5, suppress=True)
print( "Ground truth: ")
print( weights )
print( support )
print( "")

if scenario=='test1':
    centering = 0.0
    scale = 1.0
else:
    endpoint_left  = np.max( support )
    endpoint_right = np.min( support )
    centering = 0.5*( endpoint_left + endpoint_right )
    scale     = 1.1*(endpoint_left-centering)
    support   = (support-centering)/scale
    print( "Centered support: ", support)

moments_count = 7
mom_array = np.zeros( 2*moments_count + 2)
for index in range( len(mom_array) ):
    mom_array[index] = np.dot( weights, support**index)
print( f'''Computed moments from 0 to 2n+1={len(mom_array)-1}''')

# Compute
#jacobi_a, jacobi_b = freeDeconvolution.oprl.jacobi_from_moments( mom_array )
jacobi_a, jacobi_b = freeDeconvolution.jacobi_from_moments( mom_array, debug=False )
print( "a :", jacobi_a )
print( "b :", jacobi_b )
rec_support, rec_weights   = freeDeconvolution.quadrature_from_jacobi( jacobi_a, jacobi_b)
rec_support = centering + scale*rec_support
print( "Reconstructed support: ", rec_support )
print( "Reconstructed weights: ", rec_weights )
print( "" )

chebyshev_mom_array = np.zeros( 2*moments_count + 2)
chebyshev_values    = compute_chebyshev_values( support, 2*moments_count + 2)
for mom_index in range( len(chebyshev_mom_array) ):
    chebyshev_mom_array[mom_index] = np.dot( weights, chebyshev_values[mom_index] )
print( "Chebyshev moment array")
print( chebyshev_mom_array)
print( "")

## Inverse moment problem
print( "Performing inverse Chebychev moment problem...")
jacobi_a, jacobi_b = jacobi_from_chebyshev_moments( chebyshev_mom_array, debug=False )
print( "a :", jacobi_a )
print( "b :", jacobi_b )
rec_support, rec_weights   = freeDeconvolution.quadrature_from_jacobi( jacobi_a, jacobi_b)
rec_support = centering + scale*rec_support
print( "Reconstructed support: ", rec_support )
print( "Reconstructed weights: ", rec_weights)

Ground truth: 
[0.2 0.2 0.2 0.2 0.2]
[1. 2. 3. 5. 6.]

Centered support:  [-0.90909 -0.54545 -0.18182  0.54545  0.90909]
Computed moments from 0 to 2n+1=15
6-th principal minor is not positive definite
a : [-0.03636  0.07019 -0.133    0.18474 -0.26738]
b : [0.67444 0.46933 0.50673 0.37921]
Reconstructed support:  [1. 2. 3. 5. 6.]
Reconstructed weights:  [0.2 0.2 0.2 0.2 0.2]

Chebyshev moment array
[ 1.      -0.03636 -0.0438   0.02607 -0.02237 -0.0099  -0.00155  0.00299
 -0.00312 -0.00078 -0.0009   0.00018  0.00018 -0.00003  0.00008  0.     ]

Performing inverse Chebychev moment problem...
7-th principal minor is not positive definite
a : [-0.03636  0.07019 -0.133    0.18474 -0.26738 -0.44318]
b : [0.67444 0.46933 0.50673 0.37921 0.     ]
Reconstructed support:  [1.      2.      2.28125 3.      5.      6.     ]
Reconstructed weights:  [0.2 0.2 0.  0.2 0.2 0.2]


In [86]:
def compute_chebyshev_values( x, order ):
    result = []
    #
    term_0 = np.ones_like(x)*1.0
    result.append( term_0 )
    term_1 = x
    result.append( x )
    for i in range(2, order+1):
            # Compute next term in recurrence
            # Normally P_{n+1} = 2X P_n - P_{n-1}
            # But P_n = 2^{n-1} T_n
            # Hence T_{n+1} = X P_n - (1/4)*P_{n-1}
            if i==2:
                 b2 = 0.5
            else:
                 b2 = 0.25
            term_2 = x*term_1 - b2*term_0
            result.append( term_2 )
            # Shift variables
            term_0 = term_1 
            term_1 = term_2
    # end for
    return result

In [89]:
# Input: Array of 2n+1 moments from c_0=1 to c_{2n+1}
def jacobi_from_chebyshev_moments( mom_array, debug=False ):
    assert( (len(mom_array)-1) % 2 == 1 )
    mom_array = np.hstack( (mom_array, [0]))
    
    # Form Gram matrix
    # Uses the fact that
    # 2 P_n P_m = P_{n+m} + P_{|n-m|} for the (non-monic) Chebyshev
    # This leads to
    # T_n T_m = T_{n+m} + 2^{|m-n|-n-m} T_{|n-m|}
    #         = T_{n+m} + 2^{-2*min(n,m)} T_{|n-m|}
    # Except when n=m where
    # T_n^2 = T_{2n} + 2^{-2n+1}
    n = int( 0.5*(len(mom_array)-2) )
    gram_matrix = np.zeros( shape=(n+2, n+2) )
    for i in range( n+2 ):
        if i==0:
            for j in range( i+1, n+2):
                gram_matrix[i,j] = mom_array[i+j]
        else:
            for j in range( i+1, n+2):
                gram_matrix[i,j] = mom_array[i+j] + (2**(-2*i))*mom_array[j-i]
    gram_matrix = gram_matrix + gram_matrix.transpose()
    for index in range(1, n+2 ):
        gram_matrix[index, index] = mom_array[2*index] + 2**(-2*index+1)
    gram_matrix[0,0] = 1

    if debug:
        print( "Gram matrix: ", gram_matrix)
    
    # Cholesky
    # try:
    #     cholesky = scipy_cholesky( mom_matrix, lower=True )
    #     print("Cholesky passed!")
    #     print("")
    # except np.linalg.LinAlgError as err:
    #     print( err  )
    #     print("")

    # Use of LAPACK wrapper in scipy
    # https://stackoverflow.com/questions/49101574/scipy-numpy-cholesky-while-checking-if-positive-definite
    (cholesky, minor) = scipy.linalg.lapack.dpotrf( gram_matrix, lower=True  )
    if minor > 0:
        # if debug:
        #     print( f'''{minor}-th principal minor is not positive definite''')
        print( f'''{minor}-th principal minor is not positive definite''')
        gram_matrix = gram_matrix[:minor, :minor]
        cholesky   = cholesky[:minor, :minor]
        #
        n = minor - 1

    if debug:
        print( "Cholesky matrix: ")
        print( cholesky )

    mom_count = cholesky.shape[0]
    diag_indices, extra_indices = freeDeconvolution.oprl.jacobi_indices( mom_count )
    diag_cholesky = cholesky.T[diag_indices[0,:] , diag_indices[1,:]]
    extra_diag    = cholesky.T[extra_indices[0,:], extra_indices[1,:]]

    if debug:
        print( "Diagonal of Cholesky")
        print( diag_cholesky )
        print( "Extra-diagonal of Cholesky")
        print( extra_diag )
        print("")

    # Compute Jacobi
    jacobi_b = diag_cholesky[1:]/diag_cholesky[:-1]
    jacobi_b = jacobi_b[:-1]
    jacobi_a = np.zeros_like( extra_diag )
    if len(extra_diag)>0:
        jacobi_a[0] = extra_diag[0]
        x_over_y = extra_diag/diag_cholesky[:-1]
        jacobi_a[1:] = x_over_y[1:]-x_over_y[:-1]

    return jacobi_a, jacobi_b

# II. Iterative approach: Three term recurrence

NOT WORKING YET

In this approach, we avoid computing moments at all cost! Perhaps this has better stability

For the monic polynomials $P_n$:
$$ X P_n = P_{n+1} + a_n P_n + b_{n-1}^2 P_{n-1} $$

From that one deduces (after some work) that:
$$ \| P_n \| = b_1 \dots b_{n-1}, $$
which yields the recurrence for orthonormal polynomials $p_n$:
$$ X p_n = b_n p_{n+1} + a_n p_n + b_{n-1} p_{n-1} $$

In [ ]:
def polynomial_eval( coeffs, points):
    m = len(coeffs)
    powers = np.arange( 0, m, 1)
    vander = points[..., None]**powers[None, ...]
    return np.dot(vander, coeffs)

# Test
points = np.arange( 0, 5)
coefficients = [1, 0]
print( polynomial_eval(coefficients, points) )
coefficients = [1, 1]
print( polynomial_eval(coefficients, points) )


In [ ]:
# Compute OPRL
polynomials = []
norms = []

# First in the iteration: P_0 = 1
polynomials.append( [1] )
norms.append( 1 )

# Second in the iteration: P_1 = X-m_1
m1 = np.real( cauchy_integral_g_deconv( z_array ) )
polynomials.append( [-m1, 1] )
P1 = np.array( polynomials[-1] )
values_P1 = polynomial_eval(P1, z_array)
norm2_P1 = np.real( cauchy_integral_g_deconv( values_P1*values_P1 ) )
norm_P1  = np.sqrt(norm2_P1)
assert( norm2_P1 > 0)
norms.append( norm_P1 )

# Apply three term recurrence
OPRL_order = 10
jacobi_a = [ m1 ]
jacobi_b = [ ]
for n in np.arange(1, OPRL_order, 1):
    Pn  = polynomials[-1] + [0.0]
    XPn = [0.0] + polynomials[-1]
    Qn  = polynomials[-2] + [0.0, 0.0] # Short for P_{n-1}
    Pn, XPn, Qn = [ np.array(x) for x in [Pn, XPn, Qn] ]
    # Values along path
    values_Pn  = polynomial_eval(  Pn, z_array)
    values_XPn = z_array*values_Pn
    values_Qn  = polynomial_eval(  Qn, z_array)
    # Norms
    norm_Pn = norms[-1]
    norm_Qn = norms[-2]
    # Compte a_n and b_{n-1}
    a  = np.real( cauchy_integral_g_deconv( values_XPn*values_Pn ) )/( norm_Pn)
    b2 = np.real( cauchy_integral_g_deconv( values_XPn*values_Qn ) )/( norm_Qn)
    print( "n: ", n+1)
    print( "a: ", a)
    print( "b2: ", b2)
    print( "")
    if b2 < 0:
        print( "Defaulting at n =", n+1)
        print( "")
        OPRL_order = n
        #jacobi_a.append( a )
        break
    #
    b = np.sqrt(b2)
    jacobi_a.append( a )
    jacobi_b.append( b )
    norms.append( norm_Pn*b )
    # Three term recurrence
    new_P = XPn - a*Pn - b*b*Qn
    polynomials.append( list(new_P) )
# end for
print( "Jacobi coefficients:")
print( jacobi_a )
print( jacobi_b )
print( "")
print( "Norms:")
print( norms )